In [1]:
import pandas as pd
import re
import requests
from nltk.util import bigrams

import time
from datetime import datetime 

start_time = datetime.now() 

In [2]:
df = pd.read_csv('sentences_521_features_46.csv')
df

,sentence,mdd,t_depth,no_change,summarizing,bigrams,words,letters,word_avg,numbers,...,RB,CC,AT,QW,REL,PRP,IN,DTT,DT,MD
0,והנפיחות בעיניים?,3.75,4,1,0,1,2,15,7.500,0,...,0,0,0,0,0,0,0,0,0,0
1,"ככלל, כדאי להיות קשובים לגוף וכשאשר מופיעים סי...",14.82,34,0,1,25,25,130,5.200,0,...,2,2,0,0,0,0,0,0,0,0
2,הצגה עצמיתתחילה תתבקש לפרט את המחלות או הליקוי...,14.41,28,0,1,19,21,99,4.714,0,...,0,2,2,2,2,4,2,0,0,0
3,מגמת משרד החינוך הינה לשלב את הילד בעל הצרכים ...,18.42,36,0,1,25,26,126,4.846,0,...,2,2,2,0,2,0,4,2,0,0
4,"בייעוץ כזה יובאו בחשבון המסלול, משך הטיול, אופ...",19.70,43,0,1,24,24,139,5.792,0,...,0,0,0,2,4,2,0,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516,"הורים יקרים,במדריך הזה מפורט התהליך העומד בפני...",16.91,33,0,1,22,24,122,5.083,0,...,0,0,2,0,8,2,6,0,0,0
517,מהיום ניתן לקבל את חשבון הארנונה ישירות לדואר...,7.17,15,0,1,11,12,62,5.167,0,...,2,0,2,0,0,0,0,0,0,2
518,".תורם לאיכות הסביבה, קבלת חשבון ארנונה תקופתי ...",13.08,25,0,1,18,19,92,4.842,0,...,0,0,0,0,0,0,4,0,0,2
519,ניתן לקבל את חשבון הארנונה למייל במילוי טופס.,5.27,12,0,1,7,8,37,4.625,0,...,0,0,2,0,0,0,0,0,0,2


In [3]:
df2 = df.copy()
df2['sentence'] = df['sentence'].str.replace('[^\w\s]','') 
df2['sentence'] = df['sentence'].str.replace('[^א-ת ]','') 
df2['sentence']

0                                       והנפיחות בעיניים
1      ככלל כדאי להיות קשובים לגוף וכשאשר מופיעים סימ...
2      הצגה עצמיתתחילה תתבקש לפרט את המחלות או הליקוי...
3      מגמת משרד החינוך הינה לשלב את הילד בעל הצרכים ...
4      בייעוץ כזה יובאו בחשבון המסלול משך הטיול אופיו...
                             ...                        
516    הורים יקריםבמדריך הזה מפורט התהליך העומד בפניכ...
517    מהיום ניתן לקבל  את חשבון הארנונה ישירות לדואר...
518    תורם לאיכות הסביבה קבלת חשבון ארנונה תקופתי לל...
519         ניתן לקבל את חשבון הארנונה למייל במילוי טופס
520    חשבון הארנונה בדואר האלקטרוני מגיע כשהוא מאובט...
Name: sentence, Length: 521, dtype: object

In [4]:
stopwords = ['אני','את','אתה','אנחנו','אתן','אתם','הם','הן','היא','הוא','שלי','שלו','שלך','שלה','שלנו','שלכם','שלכן','שלהם','שלהן','לי',
'לו','לה','לנו','לכם','לכן','להם','להן','אותה','אותו','זה','זאת','אלה','אלו','תחת','מתחת','מעל','בין','עם','עד','נגר','על','אל','מול','של',
'אצל','כמו','אחר','אותו','בלי','לפני','אחרי','מאחורי','עלי','עליו','עליה','עליך','עלינו','עליכם','לעיכן','עליהם','עליהן','כל','כולם','כולן','כך',
'ככה','כזה','זה','זות','אותי','אותה','אותם','אותך','אותו','אותן','אותנו','ואת','את','אתכם','אתכן','איתי','איתו','איתך','איתה','איתם','איתן',
'איתנו','איתכם','איתכן','יהיה','תהיה','היתי','היתה','היה','להיות','עצמי','עצמו','עצמה','עצמם','עצמן','עצמנו','עצמהם','עצמהן','מי','מה','איפה',
'היכן','אם','לאן','איזה','מהיכן','איך','כיצד','מתי','כאשר','כש','למרות',
'לפני','אחרי','למה','מדוע','כי','יש','אין','אך','מנין','מאין','מאיפה','יכל','יכלה','יכלו','יכול','יכולה','יכולים','יכולות','יוכלו','יוכל','מסוגל',
'לא','רק','אולי','אין','לאו','אי','כלל','נגד','אם','עם','אל','אלה','אלו','אף','על','מעל','מתחת','מצד','בשביל','לבין','באמצע','בתוך','דרך',
'מבעד','באמצעות','למעלה','למטה','מחוץ','מן','לעבר','מכאן','כאן','הנה','הרי','פה','שם','אך','ברם','שוב','אבל','מבלי','בלי','מלבד','רק',
'בגלל','מכיוון','עד','אשר','ואילו','למרות','כמו','כפי','אז','אחרי','כן','לכן','לפיכך','מאד','מעט','מעטים','במידה','שוב','יותר','מדי','גם','כן',
'נו','אחר','אחרת','אחרים','אחרות','אשר','או']
len(stopwords)

211

In [5]:
num_of_stopwords = []
for s in df2.sentence:
    spl = s.split()
    n = 0
    for w in spl:
        if w in stopwords:
            n = n + 1
    num_of_stopwords.append(n)

In [6]:
df['num_of_stopwords'] = num_of_stopwords
df

,sentence,mdd,t_depth,no_change,summarizing,bigrams,words,letters,word_avg,numbers,...,CC,AT,QW,REL,PRP,IN,DTT,DT,MD,num_of_stopwords
0,והנפיחות בעיניים?,3.75,4,1,0,1,2,15,7.500,0,...,0,0,0,0,0,0,0,0,0,0
1,"ככלל, כדאי להיות קשובים לגוף וכשאשר מופיעים סי...",14.82,34,0,1,25,25,130,5.200,0,...,2,0,0,0,0,0,0,0,0,2
2,הצגה עצמיתתחילה תתבקש לפרט את המחלות או הליקוי...,14.41,28,0,1,19,21,99,4.714,0,...,2,2,2,2,4,2,0,0,0,4
3,מגמת משרד החינוך הינה לשלב את הילד בעל הצרכים ...,18.42,36,0,1,25,26,126,4.846,0,...,2,2,0,2,0,4,2,0,0,4
4,"בייעוץ כזה יובאו בחשבון המסלול, משך הטיול, אופ...",19.70,43,0,1,24,24,139,5.792,0,...,0,0,2,4,2,0,2,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516,"הורים יקרים,במדריך הזה מפורט התהליך העומד בפני...",16.91,33,0,1,22,24,122,5.083,0,...,0,2,0,8,2,6,0,0,0,2
517,מהיום ניתן לקבל את חשבון הארנונה ישירות לדואר...,7.17,15,0,1,11,12,62,5.167,0,...,0,2,0,0,0,0,0,0,2,1
518,".תורם לאיכות הסביבה, קבלת חשבון ארנונה תקופתי ...",13.08,25,0,1,18,19,92,4.842,0,...,0,0,0,0,0,4,0,0,2,0
519,ניתן לקבל את חשבון הארנונה למייל במילוי טופס.,5.27,12,0,1,7,8,37,4.625,0,...,0,2,0,0,0,0,0,0,2,1


In [7]:
df = pd.read_excel('new_sentences.xlsx')
df

,sentence
0,לא עשינו זאת מעולם.
1,זה מסובך מדי.
2,אין לנו משאבים.
3,זה לעולם לא יעבוד.
4,אין לנו מספיק זמן.
...,...
414,האם אוהבים אותו כי הוא מצחיק משעשע?
415,יש לו מסיכה של ביטחון עצמי?
416,אז יום אחד הוא בכה.
417,ולמה אתם לא בוכים?


In [8]:
l = []
for i in range(len(df)):
    l.append(0)

In [9]:
df['summarizing'] = l
df

,sentence,summarizing
0,לא עשינו זאת מעולם.,0
1,זה מסובך מדי.,0
2,אין לנו משאבים.,0
3,זה לעולם לא יעבוד.,0
4,אין לנו מספיק זמן.,0
...,...,...
414,האם אוהבים אותו כי הוא מצחיק משעשע?,0
415,יש לו מסיכה של ביטחון עצמי?,0
416,אז יום אחד הוא בכה.,0
417,ולמה אתם לא בוכים?,0


In [10]:
df.to_csv('sentences_419.csv', index = False)

In [11]:
# df = pd.concat([df, df], ignore_index=True)
# df

In [12]:
df = df.append(df, ignore_index=True)
df

,sentence,summarizing
0,לא עשינו זאת מעולם.,0
1,זה מסובך מדי.,0
2,אין לנו משאבים.,0
3,זה לעולם לא יעבוד.,0
4,אין לנו מספיק זמן.,0
...,...,...
833,האם אוהבים אותו כי הוא מצחיק משעשע?,0
834,יש לו מסיכה של ביטחון עצמי?,0
835,אז יום אחד הוא בכה.,0
836,ולמה אתם לא בוכים?,0
